In [ ]:
# Learning material: https://github.com/openai/openai-cookbook/blob/main/examples/Assistants_API_overview_python.ipynb

In [3]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
load_dotenv()

# pretty printing helper
import json
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-3.5-turbo",
)
show_json(assistant)

In [ ]:
thread = client.beta.threads.create()
show_json(thread)


In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)


In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

In [5]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [ ]:
run = wait_on_run(run, thread)
show_json(run)

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

Let's ask our Assistant to explain the result a bit further!

In [ ]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

Example

In [7]:
from openai import OpenAI


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

Notice how all of these API calls are asynchronous operations; this means we actually get async behavior in our code without the use of async libraries! (e.g. asyncio)

In [ ]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
thread2, run2 = create_thread_and_run("Could you explain linear algebra to me?")
thread3, run3 = create_thread_and_run("I don't like math. What can I do?")

# Now all Runs are executing...

In [ ]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# Thank our assistant on Thread 3 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

How do we use Code Interpreter in Assistant API to solve the same problem?

In [ ]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

In [ ]:
thread, run = create_thread_and_run(
    "Generate the first 20 fibbonaci numbers with code."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

For some use cases this may be enough – however, if we want more details on what precisely an Assistant is doing we can take a look at a Run's Steps.

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

In [ ]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

In [ ]:
MATH_ASSISTANT_ID

In [ ]:
# Upload the file
file = client.files.create(
    file=open(
        "data/cities_for_map.csv",
        "rb",
    ),
    purpose="assistants",
)
# Update Assistant
# assistant = client.beta.assistants.update(
#     MATH_ASSISTANT_ID,
#     tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
#     file_ids=[file.id],
# )

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are professional frontend data visualization developer. Please generate code that answers users questions on data",
    model="gpt-3.5-turbo",
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[file.id],
)
show_json(assistant)

In [ ]:
thread, run = create_thread_and_run(
    "Can you describe the data?"
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

Can the assistant handle two csv files?

In [8]:
city_data = client.files.create(
    file=open(
        "data/cities_for_map.csv",
        "rb",
    ),
    purpose="assistants",
)

city_data_catalogue = client.files.create(
    file=open(
        "data/catalogue.csv",
        "rb",
    ),
    purpose="assistants",
)

In [10]:
assistant = client.beta.assistants.update(
    name="Data Visualization Developer",
    instructions="You are professional data visualization developer. Please generate code that answers users questions on data",
    model="gpt-3.5-turbo",
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    assistant_id='asst_IlfxlF2eIwYoQQ4PZE3U5tBw',
    file_ids=[city_data.id,city_data_catalogue.id],
)
show_json(assistant)

{'id': 'asst_IlfxlF2eIwYoQQ4PZE3U5tBw',
 'created_at': 1713299059,
 'description': None,
 'file_ids': ['file-rOsPKMwi3BSKNFOVeO1Gnpcq',
  'file-swrBVR6MDG9QDWZsJ0zIAsaK'],
 'instructions': 'You are professional data visualization developer. Please generate code that answers users questions on data',
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': 'Data Visualization Developer',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}],
 'top_p': 1.0,
 'temperature': 1.0,
 'response_format': 'auto'}

In [ ]:
thread, run = create_thread_and_run(
    "You are given two data files. \
        cities_for_map.csv is the data that describe emissoins of cities from different kinds of wastes. catalogue.csv is the file of data catalogue,\
            which describes the column headers of the cities_for_map.csv. Please generate \
        code that can make an interactive map using 2022 emissions data. Each bubble on the map represents one city. The size of  \
        the bubble is proportional to the total emissions for the city. When users click on the ciy, it will\
             show the emissions for each type of waste as a pie chart."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

In [ ]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

In [ ]:
assistant.id

In [ ]:
thread.id

In [ ]:
run = submit_message(assistant.id, thread, "Please print code only")
run = wait_on_run(run, thread)
pretty_print(get_response(thread))